# HeatMapWithTime Citycrowd

In this example we show the most basic usage of the HeatMapWithTime plugin.

We generate a random set of points with lat/lon coordinates to draw on the map, and then move these points slowly in a random direction to simulate a time dimension. The points are aranged into a list of sets of data to draw.

In [1]:
import folium
import folium.plugins as plugins
import numpy as np


In [2]:
# -*- coding: utf-8 -*-
"""
指定一个方框的左下角和右上角经纬度坐标和切分的行数列数
返回切分后的每个小方框中心点的坐标值shape=(row,col)= [lat,lon]

读取h5文件获取shape数据

数据做维度变换做成move_data格式要求的数据
Author: elesun
"""
from __future__ import print_function
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import h5py
import time

loc_downleft = [39.7938, 116.2033]#latitude,longitude
loc_upright = [40.0403, 116.5358]
row = 32
col = 32

def area_split(loc_downleft,loc_upright,row,col):
    location = np.zeros(shape=(row,col,2),dtype=float)# 3 mean [lat,lon,val]
    #print("location_init\n",location)
    row_length = (loc_upright[0] - loc_downleft[0])/row
    col_length = (loc_upright[1] - loc_downleft[1])/col
    print("row_length",row_length)
    print("col_length",col_length)
    for i in range(row) :
        for j in range(col) :
            # location[i, j, :] = [37.9, 118.35, 1]
            location[i,j] = [loc_upright[0]-(i+0.5)*row_length,loc_downleft[1]+(j+0.5)*col_length]
    return (location)

print("#################area split######################")
location = area_split(loc_downleft,loc_upright,row,col)
#print("location\n",location)
print("location[0,0,:] = \n",location[0,0,:])
print("location.shape ",location.shape)

print("#################read small h5######################")
#h5 data shape = (48, 32, 32) = (step, h, w)
frname = "small.h5"
print("reading ",frname)
fr = h5py.File(frname, 'r')
print ("fr.keys() = ",fr.keys())
data = fr['small_data'][()]
fr.close()
print ("data.shape = ",data.shape)
print ("data.dtype = ",data.dtype)
print("max data = ", np.max(data))
print("min data = ", np.min(data))
data = (data-np.min(data))/(np.max(data)-np.min(data))
#print ("data = ",data)
print ("data[0,0,:] = \n",data[0,:,0] )

print("#################array reshape######################")
#move_data shape (48, 1024, 3) (step, samples, data)
#data shape 3 = (lat,lon,val)
#转换过程：h5 data(48, 32, 32) to (48,32*32,3),3 =(lat,lon,val)
data = data.reshape((48,32*32,1), order='C')
location = location.reshape((32*32,2), order='C')
#print("location\n",location)
temp = np.insert(data, 0, location[:,1], axis=2)
move_data = np.insert(temp, 0, location[:,0], axis=2)
print ("move_data.shape ",move_data.shape)
print ("move_data[0,0:3,:] = \n",move_data[0,0:3,:])
data2 = move_data[20:40,]
print ("data2.shape ",data2.shape)
#print ("data2[0,0:33,:] \n",data2[0,0:33,:])
data2 = data2.tolist()


#################area split######################
row_length 0.007703125000000144
col_length 0.010390624999999876
location[0,0,:] = 
 [ 40.03644844 116.20849531]
location.shape  (32, 32, 2)
#################read small h5######################
reading  small.h5
fr.keys() =  <KeysViewHDF5 ['small_data']>
data.shape =  (48, 32, 32)
data.dtype =  float64
max data =  965.0
min data =  0.0
data[0,0,:] = 
 [0.         0.         0.         0.01036269 0.00725389 0.00310881
 0.         0.00518135 0.01865285 0.00207254 0.         0.00207254
 0.03108808 0.         0.00103627 0.0134715  0.04663212 0.01243523
 0.03834197 0.0134715  0.05492228 0.02176166 0.00829016 0.00414508
 0.01761658 0.05492228 0.00621762 0.00829016 0.00207254 0.00103627
 0.         0.00103627]
#################array reshape######################
move_data.shape  (48, 1024, 3)
move_data[0,0:3,:] = 
 [[4.00364484e+01 1.16208495e+02 0.00000000e+00]
 [4.00364484e+01 1.16218886e+02 2.07253886e-02]
 [4.00364484e+01 1.16229277e+02 4.2

In [3]:
m = folium.Map([39.9, 116.35], zoom_start=11)#zoom_start small or big  tiles='stamentoner' #data1 data2
#m = folium.Map([35, 110], zoom_start=5)#zoom_start small or big  tiles='stamentoner'  #data3
hm = plugins.HeatMapWithTime(data2,radius=22) # data1 data2
#hm = plugins.HeatMapWithTime(data3,radius=8) # data3

hm.add_to(m)

m